<a href="https://colab.research.google.com/github/d-tomas/data-mining/blob/main/notebooks/data_mining_3.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Una historia con datos

## Pasos previos

In [ ]:
# Importamos las librerías de Python necesarias

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

Vamos a trabajar con un conjunto de datos en formato CSV que contiene estadísticas sobre nombres de niños y niñas en estados unidos, desde 1880 a 2010.

Cada fila contiene la siguiente información (el nombre de cada columna no aparece en el CSV):

* `name`: nombre entre 2 y 15 caracteres
* `sex`: M (niño) o F (niña)
* `births`: número de nacimientos con ese nombre en el año

In [ ]:
# Obtención del fichero CSV con los datos

!wget https://github.com/d-tomas/data-mining/raw/main/datasets/babynames.tgz
!tar xvfz babynames.tgz
!rm babynames.tgz

## Análisis inicial

In [ ]:
# A ver qué pinta tienen los datos

!head -n 10 babynames/yob1880.txt

In [ ]:
# Empezamos analizando los datos de 1880

names1880 = pd.read_csv('babynames/yob1880.txt', names=['name', 'sex', 'births'])
names1880

In [ ]:
# Número de niños y niñas que nacieron en 1880

names1880.groupby('sex')['births'].sum()

In [ ]:
# Construimos un DataFrame con toda la información de todos los años

years = range(1880, 2011)
columns = ['name', 'sex', 'births']
list_df = []

for year in years:
  path = 'babynames/yob%d.txt' % year
  df = pd.read_csv(path, names=columns)
  df['year'] = year
  list_df.append(df)

# Se concatenan todos los DataFrame en uno solo
data = pd.concat(list_df, ignore_index=True)
data

In [ ]:
# Creamos una tabla que resume el número de nacimientos, para niños y niñas, en cada año

total_births = data.pivot_table(values='births', index='year', columns='sex', aggfunc=sum)
total_births

In [ ]:
# Mostramos un diagrama de líneas para ver la evolución de nacimientos en el tiempo

plt.figure(figsize=(10, 5))
sns.lineplot(data=total_births)
plt.show()

In [ ]:
# Calculamos el porcentaje de nacimientos que tuvieron ese nombre para niños y niñas

def add_prop(group):
  group['prop'] = group['births'] / group['births'].sum()
  return group

data = data.groupby(['year', 'sex']).apply(add_prop)
data

In [ ]:
# Comprobamos que la suma de los valores anteriores es 1 para todos los años y sexo

data.groupby(['year', 'sex'])['prop'].sum()

In [ ]:
# Nos quedamos con los mil nombres más populares de niño y niña para cada año

def get_top1000(group):
    return group.sort_values(by='births', ascending=False)[:1000]

top1000 = data.groupby(['year', 'sex']).apply(get_top1000)
top1000.reset_index(inplace=True, drop=True)  # Eliminamos el índice del grupo, que no aporta nada
top1000

## Tendencia de los nombres

In [ ]:
# Creamos una tabla resumen con el número de nacimientos para los nombres más populares en el top 1000 para cada año

total_births = top1000.pivot_table(values='births', index='year', columns='name', aggfunc=sum)
total_births

In [ ]:
# Mostramos un diagrama de líneas con la evolución de cuatro de los nombres

plt.figure(figsize=(10, 5))
sns.lineplot(data=total_births[['John', 'Harry', 'Mary', 'Marilyn']])
plt.show()

## Midiendo la diversidad de nombres

In [ ]:
# Calculamos el porcentaje de nombres que supone sobre el total los 1000 primeros
# Esto muestra la diversidad de nombres que han ido apareciendo

table = top1000.pivot_table(values='prop', index='year', columns='sex', aggfunc=sum)

plt.figure(figsize=(10, 5))
sns.lineplot(data=table)
plt.show()

In [ ]:
# Nombres de niños en el año 2010

boys = top1000[top1000['sex'] == 'M']  # Nos quedamos con los nombres de niños
df = boys[boys['year'] == 2010]
df

In [ ]:
# Calculamos cuántos nombres acumulan el 50% de los nacimientos

prop_cumsum = df.sort_values(by='prop', ascending=False)['prop'].cumsum()
prop_cumsum.values.searchsorted(0.5) + 1  # Busca en qué posición de la lista caería el valor 0.5

In [ ]:
# Repetimos el cálculo anterior pero para el año 1900

df = boys[boys['year'] == 1900]
prop_cumsum = df.sort_values(by='prop', ascending=False)['prop'].cumsum()
prop_cumsum.values.searchsorted(0.5) + 1

In [ ]:
# Mostramos el porcentaje de nacimientos que acumulan los 1000 primeros nombres para cada año

top1000.groupby(['year', 'sex']).sum()

In [ ]:
# Hacemos el cálculo (cuántos nombres permiten acumular el 50% de los nacimientos)
# Se hace para todos los años y tanto para niños como para niñas

def get_quantile_count(group):
  group = group.sort_values(by='prop', ascending=False)
  return group['prop'].cumsum().values.searchsorted(0.5) + 1

diversity = top1000.groupby(['year', 'sex']).apply(get_quantile_count)
diversity = diversity.unstack('sex')
diversity

In [ ]:
# Mostramos la gráfica con cuántos nombres acumulan el 50% de nacimientos

plt.figure(figsize=(10, 5))
sns.lineplot(data=diversity)
plt.show()

## La revolución de la última letra

In [ ]:
# Creamos una tabla que resume, por sexo y años, el número de niños y niñas cuyo nombre acaba en cada letra

get_last_letter = lambda x: x[-1]
last_letters = data['name'].map(get_last_letter)  # Sacamos la última letra de cada nombre
table = data.pivot_table(values='births', index=last_letters, columns=['sex', 'year'], aggfunc=sum)
table

In [ ]:
# Nos quedamos con un subconjunto de tres años

subtable = table.reindex(columns=[1910, 1960, 2010], level='year')
subtable

In [ ]:
# Sustituimos el número de ocurrencias de cada letra por su probabilidad de ocurrencia 

letter_prop = subtable / subtable.sum()
letter_prop

In [ ]:
# Cómo han evolucionado los nombres terminados en cada letra para cada uno de los años

plt.figure(figsize=(10, 5))
sns.lineplot(data=letter_prop['M'])  # Nombres de niños
plt.show()
plt.figure(figsize=(10, 5))
sns.lineplot(data=letter_prop['F'])  # Nombres de niñas
plt.show()

In [ ]:
# Vemos como evolucionan tres de las letras para los niños

letter_prop = table / table.sum()  # Calculamos el porcentaje que representa cada letra en cada año
dny_ts = letter_prop.loc[['d', 'n', 'y'], 'M'].T
dny_ts

In [ ]:
# La gráfica muestra cómo evolucionan las terminaciones 'd', 'n' e 'y' a lo largo de los años

plt.figure(figsize=(10, 5))
sns.lineplot(data=dny_ts)
plt.show()

## Nombres que cambiaron de sexo

In [ ]:
# Obtenemos todos los nombres que contienen 'lesl'

all_names = pd.Series(top1000['name'].unique())  # Lista con todos los nombres en el top 1000
lesley_like = all_names[all_names.str.lower().str.contains('lesl')]  # Nombres que contienen 'lesl'
lesley_like

In [ ]:
# Contamos el número de nacimientos cuyo nombre empieza por 'lesl'

filtered = top1000[top1000['name'].isin(lesley_like)]
filtered.groupby('name')['births'].sum()

In [ ]:
# Calculamos, para los nombres que empiezan por 'lesl', qué porcentaje son niños y cuál niñas para cada año

table = filtered.pivot_table(values='births', index='year', columns='sex', aggfunc='sum')
table = table.div(table.sum(1), axis=0)  # Obtiene el porcentaje de ocurrencias entre niños y niñas
table

In [ ]:
# Mostramos cómo han evolucionado los nombres 'lesl' para niños y niñas con los años

plt.figure(figsize=(10, 5))
sns.lineplot(data=table)
plt.show()

## Referencias

* [Python for Data Analysis](https://github.com/wesm/pydata-book/blob/2nd-edition/ch14.ipynb)